In [7]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA,SparsePCA,IncrementalPCA,TruncatedSVD,FastICA,MiniBatchSparsePCA,FactorAnalysis
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import shapiro,f_oneway,kruskal

data = pd.read_csv("Electrical Grid.csv")

data_x = data.drop(['stab','stabf'],axis=1)
data_y = data['stab']

scaler = MinMaxScaler()
scaled_x = scaler.fit_transform(data_x)
scaled_x = pd.DataFrame(scaled_x, columns=data_x.columns)

In [8]:
depth_data = pd.read_csv("depth.csv")
tree_data = pd.read_csv("tree.csv")
max_feature_data = pd.read_csv("max feature.csv")
bootstrap_size_data = pd.read_csv("bootstrap size.csv")
batch_data = pd.read_csv("batch size.csv")
iteration_data = pd.read_csv("iteration.csv")

best_parameter = pd.read_csv("bast parameter.csv")
anova_data = pd.read_csv("anova result.csv")

In [9]:
def normal_test(data):
    test_stat,p_value = shapiro(data)
    return np.round(p_value,3)

In [10]:
spca_rotf_paremater_set = {"trees": [100,300, 500, 700], "sampling_sizes": [0.5, 0.7, 0.9], 'depths': [10, 20, 30, 40, 50],
               'batch': [3, 5, 7, 9], 'iterations ': [100, 300, 500, 700]}

# SPCA-RotF

In [5]:
def get_random_subset(iterable,k):
    subsets = []
    iteration = 0
    np.random.shuffle(iterable)
    subset = 0
    limit = len(iterable)/k
    while iteration < limit:
        if k <= len(iterable):
            subset = k
        else:
            subset = len(iterable)
        subsets.append(iterable[-subset:])
        del iterable[-subset:]
        iteration+=1
    return subsets

def Rotation_Forest_SPCA(X, Y, test_x, max_depth, size, n_trees, k, batch, iter):
    r_matrices, models = [], []

    for tree in range(n_trees):
        feature_index = list(range(X.shape[1]))
        # 每個子集有k個特徵，每個子集特徵不重複 #將訓練集中的屬性拆分為大小相等的 K 個非重疊子集。
        k_subset = get_random_subset(feature_index, k)
        rotation_matrix = np.zeros(
            (X.shape[1], X.shape[1]), dtype=float)  # 591*591大小的矩陣

        for each_subset in k_subset:
            pca = MiniBatchSparsePCA(batch_size=batch, n_iter=iter)
            X_train, X_test, y_train, y_test = train_test_split(
                X, Y, train_size=size)
            X_subset = X_train.iloc[:, each_subset]
            pca.fit(X_subset)
            for i in range(0, len(pca.components_)):
                for j in range(0, len(pca.components_)):
                    rotation_matrix[each_subset[i],each_subset[j]] = pca.components_[i, j]
        x_transformed = X.dot(rotation_matrix)
        model = DecisionTreeRegressor(max_depth=max_depth)
        model.fit(x_transformed, Y)
        models.append(model)  # 存放每個樹的模型
        r_matrices.append(rotation_matrix)  # 存放每個樹的旋轉矩陣

    return models, r_matrices

def model_predict(models,r_matrices,x):
    predicted_ys = []
    for i,model in enumerate(models):
        x_mod =  x.dot(r_matrices[i])  
        predicted_y = model.predict(x_mod)
        predicted_ys.append(predicted_y)
        
    final_results = []
    for i in range(len(predicted_ys[0])): #總共要預測的Y有幾個 #2000個預測值
        predict_result = [] #存放每棵樹的預測值 #100棵樹有100個預測值
        for j in range(len(predicted_ys)): #每棵樹的預測值 #100棵樹
            predict_result.append( (predicted_ys[j][i]) )
        final_results.append(np.mean(predict_result))

    return final_results


def SPCA_RotF_kfold(tree, depth, max_sample, batch_size, iteration,seed):
    RMSE_set_spca_rotf = []
    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    # split()  method generate indices to split data intSo training and test set.
    for train_index, valid_index in kf.split(scaled_x, data_y):
        # print('fold',cnt)
        train_x = scaled_x.iloc[train_index, :]
        train_y = data.iloc[train_index, :]['stab']
        valid_x = scaled_x.iloc[valid_index, :]
        valid_y = data.iloc[valid_index, :]['stab']

        models, r_matrices = Rotation_Forest_SPCA(X=train_x, Y=train_y, test_x=valid_x, max_depth=depth, size=max_sample,
                                                  n_trees=tree, k=3, batch=batch_size, iter=iteration)
        spca_rot_pred = model_predict(models, r_matrices, valid_x)

        RMSE_set_spca_rotf.append(
            np.sqrt(mean_squared_error(valid_y, spca_rot_pred)))

    mean_rmse = np.mean(RMSE_set_spca_rotf)
    return RMSE_set_spca_rotf

In [8]:
num = 2

for param_name, parameter in spca_rotf_paremater_set.items():
    print('================',param_name,'================')
    if param_name == 'trees':
        tree_rmse  = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100
        for para in parameter:
            parm_set_2 = SPCA_RotF_kfold(tree=para, depth=30, max_sample=0.7, batch_size=5, iteration=100,seed = 47 ) #23 31 47 52
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的樹參數
            if np.mean(parm_set_2) < best_rmse:
              best_tree = para
              best_rmse = np.mean(parm_set_2)
            tree_rmse.append(parm_set_2)
            
            # 紀錄每個參數的常態檢定p-value
            tree_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)

        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(tree_rmse[0],tree_rmse[1],tree_rmse[2],tree_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(tree_rmse[0],tree_rmse[1],tree_rmse[2],tree_rmse[3])
          print('f_oneway:',test_stat,anova_p)

        anova_data['trees'].iloc[num] = np.round(anova_p,3)    # 紀錄該參數的anova result
        best_parameter['trees'].iloc[num] = best_tree   # 紀錄該參數的最佳參數
        tree_result = [np.mean(tree_rmse[0]),np.mean(tree_rmse[1]),np.mean(tree_rmse[2]),np.mean(tree_rmse[3])]   # 紀錄每個參數之平均rmse

########################################################################################################################################

    elif param_name == 'sampling_sizes':
        sample_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100
        for para in parameter:
            parm_set_2 = SPCA_RotF_kfold(tree=100, depth=30, max_sample=para, batch_size=5, iteration=100,seed = 47 )
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            sample_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            bootstrap_size_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(sample_rmse[0],sample_rmse[1],sample_rmse[2])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(sample_rmse[0],sample_rmse[1],sample_rmse[2])
          print('f_oneway:',test_stat,anova_p)

        anova_data['bootstrap size'].iloc[num] = np.round(anova_p,3)

        best_parameter['bootstrap size'].iloc[num] = best_para   # 紀錄該參數的最佳參數
        bootstrap_result = [np.mean(sample_rmse[0]),np.mean(sample_rmse[1]),np.mean(sample_rmse[2])]  # 紀錄每個參數之平均rmse

########################################################################################################################################

    elif param_name == 'depths':
        depth_rmse = []
        best_rmse,best_para = 100,1000
        for para in parameter:
            parm_set_2 = SPCA_RotF_kfold(tree=100, depth=para, max_sample=0.7, batch_size=5, iteration=100,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            depth_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            depth_data[str(para)].iloc[num] = normal_test(parm_set_2)

        print(f_oneway(depth_rmse[0],depth_rmse[1],depth_rmse[2],depth_rmse[3],depth_rmse[4]))

        # 紀錄該參數的anova result
        test_stat,anova_p = f_oneway(depth_rmse[0],depth_rmse[1],depth_rmse[2],depth_rmse[3],depth_rmse[4])
        anova_data['max depth'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['max depth'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        depth_result = [np.mean(depth_rmse[0]),np.mean(depth_rmse[1]),np.mean(depth_rmse[2]),np.mean(depth_rmse[3]),np.mean(depth_rmse[4])]

########################################################################################################################################

    elif param_name == 'batch':
        batch_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100

        for para in parameter:
            parm_set_2 = SPCA_RotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=para, iteration=100,seed = 42)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            batch_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            batch_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(batch_rmse[0],batch_rmse[1],batch_rmse[2],batch_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(batch_rmse[0],batch_rmse[1],batch_rmse[2],batch_rmse[3])
          print('f_oneway:',test_stat,anova_p)

        # 紀錄該參數的anova result
        anova_data['batch size'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['batch size'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        batch_result = [np.mean(batch_rmse[0]),np.mean(batch_rmse[1]),np.mean(batch_rmse[2]),np.mean(batch_rmse[3])]

########################################################################################################################################

    elif param_name == 'iterations':
        iter_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 1000
        for para in parameter:
            parm_set_2 = SPCA_RotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=5, iteration=para ,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            iter_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            iteration_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('f_oneway:',test_stat,anova_p)
  
        # 紀錄該參數的anova result
        anova_data['iteration'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['iteration'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        iter_result = [np.mean(iter_rmse[0]),np.mean(iter_rmse[1]),np.mean(iter_rmse[2]),np.mean(iter_rmse[3])]

print(tree_result)
print(bootstrap_result)
print(depth_result)
print(batch_result)
print(iter_result)

================ trees ================
100 0.8 [0.011186624345034504, 0.011333951107553706, 0.011055273252098544, 0.01098110576681369, 0.010937378909094006, 0.010607870175775882, 0.01153450973717361, 0.01123460964522041, 0.010710483239176543, 0.010664204132734017]
300 0.432 [0.010942888142561101, 0.011027614554259681, 0.010918216055917513, 0.0107857859202755, 0.010927071421154868, 0.010383603507246831, 0.011252670571592412, 0.011158081630928525, 0.010797298576736803, 0.010806760365062841]
500 0.293 [0.010974117765023661, 0.011044447325856657, 0.010913377699527202, 0.010787671341349574, 0.010871551664656982, 0.01032419892841868, 0.011175038666316198, 0.010892385778770882, 0.010718452405726593, 0.010782441566491881]
700 0.385 [0.010966610259141746, 0.011080569085347788, 0.010940861438984126, 0.010773119223858504, 0.010844072599316449, 0.010285777368028246, 0.011211361832527014, 0.0110754798170548, 0.010696167786797672, 0.010798684586321817]
f_oneway: 0.935390133992195 0.4336337592335245

NameError: name 'iter_result' is not defined

In [46]:
num = 2
for param_name, parameter in spca_rotf_paremater_set.items():
    if param_name == 'iterations ':
        print('================ iteration ================')
        iter_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 1000
        for para in parameter:
            parm_set_2 = SPCA_RotF_kfold(tree=100, depth=30, max_sample=0.9, batch_size=5, iteration=para ,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            iter_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            iteration_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('f_oneway:',test_stat,anova_p)
  
        # 紀錄該參數的anova result
        anova_data['iteration'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['iteration'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        iter_result = [np.mean(iter_rmse[0]),np.mean(iter_rmse[1]),np.mean(iter_rmse[2]),np.mean(iter_rmse[3])]

print(iter_result)

================ iteration ================
100 0.69 [0.011147885303628077, 0.011372689537538914, 0.011303127557971344, 0.011192143219290802, 0.011200087303894334, 0.010684656142861269, 0.011499547612180697, 0.011274659132133446, 0.010987677973412432, 0.011053465881325483]
300 0.408 [0.011216239910345078, 0.011251427307664244, 0.01094052784430253, 0.010732109475539563, 0.010978267981822599, 0.01054604320034549, 0.011245480294138823, 0.011134253418070983, 0.010887467654550191, 0.010688824050071497]
500 0.681 [0.011084742461422532, 0.011287303132342911, 0.010885884865883629, 0.010785051443319994, 0.011042375075701027, 0.010532010446870322, 0.011281829428372867, 0.01106950498775395, 0.010760789836102893, 0.010675496515466211]
700 0.094 [0.01097544938079634, 0.010980191382207545, 0.01097553939325251, 0.010801627826532847, 0.010832794056824295, 0.010262797084308203, 0.011239652308964426, 0.011017296379323317, 0.01074208580475375, 0.010842301790285462]
f_oneway: 2.8039578992550367 0.05355024

# SPCA-TWRotF

In [6]:
def weighted_average(prediction_set,oobrmse):
    final_results = []
    for i in range(len(prediction_set[0])): #總共要預測的Y有幾個 #2000個預測值
        predict_result = [] #存放每棵樹的預測值 #100棵樹有100個預測值
        final_result = 0
        for j in range(len(prediction_set)): #每棵樹的預測值 #100棵樹
            predict_result.append( (prediction_set[j][i]) )

        for k in range(len(predict_result)): #加權預測值
            final_result =  final_result + predict_result[k] * oobrmse[k]
        final_results.append(final_result)
    return final_results

def Tree_Weighting_Rotation_Forest_SPCA(X , Y, test_x, max_depth, size, n_trees, k, batch, iter):
    strength_set = []
    Prediction_set = []
    OOB_MSE = []
    r_matrices , models = [],[]
    for tree in range(n_trees):
        feature_index = list(range(X.shape[1]))
        k_subset = get_random_subset(feature_index,k) #每個子集有k個特徵，每個子集特徵不重複 #將訓練集中的屬性拆分為大小相等的 K 個非重疊子集。
        rotation_matrix = np.zeros((X.shape[1],X.shape[1]),dtype=float) #591*591大小的矩陣
        X_train, X_valid, y_train, y_valid = train_test_split(X, Y, train_size = size)

        for each_subset in k_subset:
            pca = MiniBatchSparsePCA(batch_size = batch ,n_iter = iter )
            x_train,_,_,_ = train_test_split(X_train, y_train, train_size = 0.7)
            X_subset = x_train.iloc[:,each_subset]
            pca.fit(X_subset)
            for i in range(0,len(pca.components_)):
                for j in range(0,len(pca.components_)):
                    rotation_matrix[ each_subset[i],each_subset[j] ] = pca.components_[i,j]

        x_transformed = X_train.dot(rotation_matrix)
        model = DecisionTreeRegressor(max_depth = max_depth).fit(x_transformed,y_train)

        x_valid_transformed = X_valid.dot(rotation_matrix)
        valid_prediction = model.predict(x_valid_transformed)

        models.append(model) #存放每個樹的模型
        r_matrices.append(rotation_matrix) #存放每個樹的旋轉矩陣

        OOB_MSE.append(mean_squared_error(y_valid,valid_prediction) )
        
    oob_mse_prop = OOB_MSE/np.sum(OOB_MSE)
    
    predicted_ys = [] #測試階段預測
    for i,model in enumerate(models): 
        x_mod =  test_x.dot(r_matrices[i])  
        predicted_y = model.predict(x_mod)
        predicted_ys.append(predicted_y)

    weighted_result = weighted_average(predicted_ys,oob_mse_prop)
    return weighted_result

def SPCA_TWRotF_kfold(tree, depth, max_sample, batch_size, iteration,seed):
    RMSE_set_spca_twrotf = []
    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    # split()  method generate indices to split data intSo training and test set.
    for train_index, valid_index in kf.split(scaled_x, data_y):
        # print('fold',cnt)
        train_x = scaled_x.iloc[train_index, :]
        train_y = data.iloc[train_index, :]['stab']
        valid_x = scaled_x.iloc[valid_index, :]
        valid_y = data.iloc[valid_index, :]['stab']

        TWRotF_pred = Tree_Weighting_Rotation_Forest_SPCA(X=train_x, Y=train_y, test_x=valid_x, max_depth=depth, size=max_sample,
                                                  n_trees=tree, k=3, batch=batch_size, iter=iteration)
        RMSE_set_spca_twrotf.append(np.sqrt(mean_squared_error(valid_y, TWRotF_pred)))

    mean_rmse = np.mean(RMSE_set_spca_twrotf)
    return RMSE_set_spca_twrotf

In [8]:
num = 8

for param_name, parameter in spca_rotf_paremater_set.items():
    print('================',param_name,'================')
    if param_name == 'trees':
        tree_rmse  = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100
        for para in parameter:
            parm_set_2 = SPCA_TWRotF_kfold(tree=para, depth=30, max_sample=0.7, batch_size=5, iteration=100,seed = 47 ) #23 31 47 52
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的樹參數
            if np.mean(parm_set_2) < best_rmse:
              best_tree = para
              best_rmse = np.mean(parm_set_2)
            tree_rmse.append(parm_set_2)
            
            # 紀錄每個參數的常態檢定p-value
            tree_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)

        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(tree_rmse[0],tree_rmse[1],tree_rmse[2],tree_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(tree_rmse[0],tree_rmse[1],tree_rmse[2],tree_rmse[3])
          print('f_oneway:',test_stat,anova_p)

        anova_data['trees'].iloc[num] = np.round(anova_p,3)    # 紀錄該參數的anova result
        best_parameter['trees'].iloc[num] = best_tree   # 紀錄該參數的最佳參數
        tree_result = [np.mean(tree_rmse[0]),np.mean(tree_rmse[1]),np.mean(tree_rmse[2]),np.mean(tree_rmse[3])]   # 紀錄每個參數之平均rmse

########################################################################################################################################

    elif param_name == 'sampling_sizes':
        sample_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100
        for para in parameter:
            parm_set_2 = SPCA_TWRotF_kfold(tree=100, depth=30, max_sample=para, batch_size=5, iteration=100,seed = 47 )
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            sample_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            bootstrap_size_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(sample_rmse[0],sample_rmse[1],sample_rmse[2])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(sample_rmse[0],sample_rmse[1],sample_rmse[2])
          print('f_oneway:',test_stat,anova_p)

        anova_data['bootstrap size'].iloc[num] = np.round(anova_p,3)

        best_parameter['bootstrap size'].iloc[num] = best_para   # 紀錄該參數的最佳參數
        bootstrap_result = [np.mean(sample_rmse[0]),np.mean(sample_rmse[1]),np.mean(sample_rmse[2])]  # 紀錄每個參數之平均rmse

########################################################################################################################################

    elif param_name == 'depths':
        depth_rmse = []
        best_rmse,best_para = 100,1000
        for para in parameter:
            parm_set_2 = SPCA_TWRotF_kfold(tree=100, depth=para, max_sample=0.7, batch_size=5, iteration=100,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            depth_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            depth_data[str(para)].iloc[num] = normal_test(parm_set_2)

        print(f_oneway(depth_rmse[0],depth_rmse[1],depth_rmse[2],depth_rmse[3],depth_rmse[4]))

        # 紀錄該參數的anova result
        test_stat,anova_p = f_oneway(depth_rmse[0],depth_rmse[1],depth_rmse[2],depth_rmse[3],depth_rmse[4])
        anova_data['max depth'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['max depth'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        depth_result = [np.mean(depth_rmse[0]),np.mean(depth_rmse[1]),np.mean(depth_rmse[2]),np.mean(depth_rmse[3]),np.mean(depth_rmse[4])]

########################################################################################################################################

    elif param_name == 'batch':
        batch_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100

        for para in parameter:
            parm_set_2 = SPCA_TWRotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=para, iteration=100,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            batch_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            batch_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(batch_rmse[0],batch_rmse[1],batch_rmse[2],batch_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(batch_rmse[0],batch_rmse[1],batch_rmse[2],batch_rmse[3])
          print('f_oneway:',test_stat,anova_p)

        # 紀錄該參數的anova result
        anova_data['batch size'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['batch size'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        batch_result = [np.mean(batch_rmse[0]),np.mean(batch_rmse[1]),np.mean(batch_rmse[2]),np.mean(batch_rmse[3])]

########################################################################################################################################

    elif param_name == 'iterations':
        iter_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 1000
        for para in parameter:
            parm_set_2 = SPCA_TWRotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=5, iteration=para ,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            iter_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            iteration_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('f_oneway:',test_stat,anova_p)
  
        # 紀錄該參數的anova result
        anova_data['iteration'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['iteration'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        iter_result = [np.mean(iter_rmse[0]),np.mean(iter_rmse[1]),np.mean(iter_rmse[2]),np.mean(iter_rmse[3])]

print(tree_result)
print(bootstrap_result)
print(depth_result)
print(batch_result)
print(iter_result)

================ trees ================
100 0.456 [0.011524498632911533, 0.01159982630192036, 0.011359330294501906, 0.011212503301501335, 0.011362260171584381, 0.010942910351022754, 0.011653697857300355, 0.011653980082687409, 0.011198331667814466, 0.011160404610067084]
300 0.446 [0.01139551290419669, 0.011356265881226497, 0.011251592480656446, 0.011078512408413115, 0.011255621268979575, 0.010610614421725548, 0.011600346540866072, 0.011342124128683661, 0.01103074069285751, 0.011087178627304404]
500 0.439 [0.011318791375520608, 0.011341681323907126, 0.011260701737233295, 0.011007948196829645, 0.0111497671736165, 0.01062556049330323, 0.011464677967662087, 0.011378893632326588, 0.01093169321143305, 0.011121635448351654]
700 0.779 [0.011329723106512347, 0.011392392663371302, 0.011237320966428769, 0.011090308970904142, 0.011176903612764815, 0.010721828551404293, 0.011492540291236891, 0.01135539271182622, 0.010991411273634453, 0.010982856061300795]
f_oneway: 1.460096242197034 0.24168219353861

NameError: name 'iter_result' is not defined

In [14]:
num = 2

for i in range(20):
    print('random state = ',47,': ================ iteration ================')
    iter_rmse = []
    best_rmse,best_para = 100,1000
    min_normality_p = 1000
    for para in [100,300,500,700]:
        parm_set_2 =  SPCA_TWRotF_kfold(tree=50, depth=None, max_sample=0.7, batch_size=5, iteration=para ,seed = 47)
        print(para,normal_test(parm_set_2),parm_set_2)

        # 找出最好的參數
        if np.mean(parm_set_2) < best_rmse:
            best_para = para
            best_rmse = np.mean(parm_set_2)
        iter_rmse.append(parm_set_2)

        # 紀錄每個參數的常態檢定p-value
        iteration_data[str(para)].iloc[num] = normal_test(parm_set_2)

        if normal_test(parm_set_2) < min_normality_p:
            min_normality_p = normal_test(parm_set_2) 
        
    if min_normality_p < 0.05 :
        test_stat,anova_p = kruskal(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
        print('kruskal_oneway:',test_stat,anova_p)
    else:
        test_stat,anova_p = f_oneway(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
        print('f_oneway:',test_stat,anova_p)

    if anova_p <= 0.05:
        anova_data['iteration'].iloc[num] = np.round(anova_p,3)
        best_parameter['iteration'].iloc[num] = best_para   # 紀錄該參數的最佳參數
        iter_result = [np.mean(iter_rmse[0]),np.mean(iter_rmse[1]),np.mean(iter_rmse[2]),np.mean(iter_rmse[3])]
        print(iter_result)
        break

random state =  47 : ================ iteration ================
100 0.042 [0.01162198898805917, 0.01191550406946266, 0.011519893027761758, 0.011411575321668086, 0.011637321436554358, 0.010798128852089996, 0.011953012948457017, 0.011542336511248497, 0.011599241044184385, 0.01153195037323755]
300 0.231 [0.011332838273688655, 0.011759090245353945, 0.011274231920924789, 0.011247333940802835, 0.011484662248621417, 0.011154091163074418, 0.011899305644576736, 0.011351407406793705, 0.011082977405697192, 0.011374791250465687]
500 0.162 [0.011507733663972567, 0.011368413495875201, 0.011458777802412645, 0.01119737026017333, 0.01137516724694388, 0.010928217073694005, 0.011410535640960022, 0.011406782242710146, 0.01128906939957313, 0.011182728190300941]
700 0.056 [0.011323220034012827, 0.011800347469896987, 0.011325741684483795, 0.011183612793048157, 0.011088559948887841, 0.01108910789156686, 0.011529982455467795, 0.01135083792299096, 0.01112751353329435, 0.011110366060071286]
kruskal_oneway: 9.42

# SPCA-SRotF

In [7]:
def Strength_Rotation_Forest_SPCA(X , Y, test_x, max_depth, size, n_trees, k,batch,iter):
    strength_set = []
    Prediction_set = []
    r_matrices , models = [],[]
    for tree in range(n_trees):
        feature_index = list(range(X.shape[1]))
        k_subset = get_random_subset(feature_index,k) #每個子集有k個特徵，每個子集特徵不重複 #將訓練集中的屬性拆分為大小相等的 K 個非重疊子集。
        rotation_matrix = np.zeros((X.shape[1],X.shape[1]),dtype=float) #591*591大小的矩陣
        X_train, X_valid, y_train, y_valid = train_test_split(X, Y, train_size = size)

        for each_subset in k_subset:
            pca = MiniBatchSparsePCA(batch_size = batch ,n_iter = iter )
            x_train,_,_,_ = train_test_split(X_train, y_train, train_size = 0.75)
            X_subset = x_train.iloc[:,each_subset]
            pca.fit(X_subset)
            for i in range(0,len(pca.components_)):
                for j in range(0,len(pca.components_)):
                    rotation_matrix[ each_subset[i],each_subset[j] ] = pca.components_[i,j]

        x_transformed = X_train.dot(rotation_matrix)
        model = DecisionTreeRegressor(max_depth = max_depth).fit(x_transformed,y_train)

        x_valid_transformed = X_valid.dot(rotation_matrix)
        valid_prediction = model.predict(x_valid_transformed)

        models.append(model) #存放每個樹的模型
        r_matrices.append(rotation_matrix) #存放每個樹的旋轉矩陣

        confidence = []
        margin = np.abs(valid_prediction - y_valid)
        for j in range(len(margin)):
          confidence.append(1/ math.exp(margin.values[j]))
        strength = np.sum(confidence)/len(confidence)
        strength_set.append(strength)
    
    predicted_ys = [] #測試階段預測
    for i,model in enumerate(models): 
        x_mod =  test_x.dot(r_matrices[i])  
        predicted_y = model.predict(x_mod)
        predicted_ys.append(predicted_y)
    
    final_result = []
    for i in range(len(predicted_ys[0])):
      predict_result = [] #存放每棵樹的預測值 #100棵樹有100個預測值
      for j in range(len(predicted_ys)):
        predict_result.append( (predicted_ys[j][i]) )
      strength_predict_result = np.array(predict_result) * np.array(strength_set) #每顆樹的預測值*每個樹的strength
      final_result.append( np.mean(strength_predict_result)  ) #存放最後的2000個預測值

    return final_result

def SPCA_SRotF_kfold(tree, depth, max_sample, batch_size, iteration,seed):
    RMSE_set_spca_srotf = []
    # split()  method generate indices to split data intSo training and test set.
    kf = KFold(n_splits=10, shuffle=True, random_state=seed)
    for train_index, valid_index in kf.split(scaled_x, data_y):
        # print('fold',cnt)
        train_x = scaled_x.iloc[train_index, :]
        train_y = data.iloc[train_index, :]['stab']
        valid_x = scaled_x.iloc[valid_index, :]
        valid_y = data.iloc[valid_index, :]['stab']

        TWRotF_pred = Strength_Rotation_Forest_SPCA(X=train_x, Y=train_y, test_x=valid_x, max_depth=depth, size=max_sample,
                                                  n_trees=tree, k=3, batch=batch_size, iter=iteration)

        RMSE_set_spca_srotf.append(np.sqrt(mean_squared_error(valid_y, TWRotF_pred)))

    mean_rmse = np.mean(RMSE_set_spca_srotf)
    return RMSE_set_spca_srotf

In [8]:
num = 5

for param_name, parameter in spca_rotf_paremater_set.items():
    print('================',param_name,'================')
    if param_name == 'trees':
        tree_rmse  = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100
        for para in parameter:
            parm_set_2 = SPCA_SRotF_kfold(tree=para, depth=30, max_sample=0.7, batch_size=5, iteration=100,seed = 47 ) #23 31 47 52
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的樹參數
            if np.mean(parm_set_2) < best_rmse:
              best_tree = para
              best_rmse = np.mean(parm_set_2)
            tree_rmse.append(parm_set_2)
            
            # 紀錄每個參數的常態檢定p-value
            tree_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)

        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(tree_rmse[0],tree_rmse[1],tree_rmse[2],tree_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(tree_rmse[0],tree_rmse[1],tree_rmse[2],tree_rmse[3])
          print('f_oneway:',test_stat,anova_p)

        anova_data['trees'].iloc[num] = np.round(anova_p,3)    # 紀錄該參數的anova result
        best_parameter['trees'].iloc[num] = best_tree   # 紀錄該參數的最佳參數
        tree_result = [np.mean(tree_rmse[0]),np.mean(tree_rmse[1]),np.mean(tree_rmse[2]),np.mean(tree_rmse[3])]   # 紀錄每個參數之平均rmse

########################################################################################################################################

    elif param_name == 'sampling_sizes':
        sample_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100
        for para in parameter:
            parm_set_2 = SPCA_SRotF_kfold(tree=100, depth=30, max_sample=para, batch_size=5, iteration=100,seed = 47 )
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            sample_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            bootstrap_size_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(sample_rmse[0],sample_rmse[1],sample_rmse[2])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(sample_rmse[0],sample_rmse[1],sample_rmse[2])
          print('f_oneway:',test_stat,anova_p)

        anova_data['bootstrap size'].iloc[num] = np.round(anova_p,3)

        best_parameter['bootstrap size'].iloc[num] = best_para   # 紀錄該參數的最佳參數
        bootstrap_result = [np.mean(sample_rmse[0]),np.mean(sample_rmse[1]),np.mean(sample_rmse[2])]  # 紀錄每個參數之平均rmse

########################################################################################################################################

    elif param_name == 'depths':
        depth_rmse = []
        best_rmse,best_para = 100,1000
        for para in parameter:
            parm_set_2 = SPCA_SRotF_kfold(tree=100, depth=para, max_sample=0.7, batch_size=5, iteration=100,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            depth_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            depth_data[str(para)].iloc[num] = normal_test(parm_set_2)

        print(f_oneway(depth_rmse[0],depth_rmse[1],depth_rmse[2],depth_rmse[3],depth_rmse[4]))

        # 紀錄該參數的anova result
        test_stat,anova_p = f_oneway(depth_rmse[0],depth_rmse[1],depth_rmse[2],depth_rmse[3],depth_rmse[4])
        anova_data['max depth'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['max depth'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        depth_result = [np.mean(depth_rmse[0]),np.mean(depth_rmse[1]),np.mean(depth_rmse[2]),np.mean(depth_rmse[3]),np.mean(depth_rmse[4])]

########################################################################################################################################

    elif param_name == 'batch':
        batch_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 100

        for para in parameter:
            parm_set_2 = SPCA_SRotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=para, iteration=100,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            batch_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            batch_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(batch_rmse[0],batch_rmse[1],batch_rmse[2],batch_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(batch_rmse[0],batch_rmse[1],batch_rmse[2],batch_rmse[3])
          print('f_oneway:',test_stat,anova_p)

        # 紀錄該參數的anova result
        anova_data['batch size'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['batch size'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        batch_result = [np.mean(batch_rmse[0]),np.mean(batch_rmse[1]),np.mean(batch_rmse[2]),np.mean(batch_rmse[3])]

########################################################################################################################################

    elif param_name == 'iterations':
        iter_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 1000
        for para in parameter:
            parm_set_2 = SPCA_SRotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=5, iteration=para ,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            iter_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            iteration_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('f_oneway:',test_stat,anova_p)
  
        # 紀錄該參數的anova result
        anova_data['iteration'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['iteration'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        iter_result = [np.mean(iter_rmse[0]),np.mean(iter_rmse[1]),np.mean(iter_rmse[2]),np.mean(iter_rmse[3])]

print(tree_result)
print(bootstrap_result)
print(depth_result)
print(batch_result)
print(iter_result)

================ trees ================
100 0.674 [0.01164559911947431, 0.011721644215957314, 0.011488529421885524, 0.011497863616513402, 0.011556902836328242, 0.011045499697625376, 0.012020788614432643, 0.011794691835829901, 0.011585472736810137, 0.01143552821094244]
300 0.275 [0.011574336876995895, 0.01154729480343087, 0.011496072200161097, 0.011244824685724481, 0.011498566938590324, 0.010957026851396769, 0.011680425593319528, 0.011681613703916639, 0.011264361095076425, 0.011223418004640074]
500 0.365 [0.011590782585491557, 0.011620524317368793, 0.011446187980524823, 0.011278781907467792, 0.011425421750300417, 0.01092507994442745, 0.011698832292960105, 0.011616337244897171, 0.011253581813123324, 0.011317098477347489]
700 0.757 [0.011602959552305923, 0.011482844813323026, 0.011494688204622456, 0.01130837997782688, 0.011400231893887263, 0.010970747421972748, 0.011705543504106606, 0.011641944916425305, 0.01132372061381926, 0.011232533133378299]
f_oneway: 1.1844039150284198 0.32932204742

NameError: name 'iter_result' is not defined

In [ ]:
num = 2
for param_name, parameter in spca_rotf_paremater_set.items():
    if param_name == 'iterations ':
        print('================ iteration ================')
        iter_rmse = []
        best_rmse,best_para = 100,1000
        min_normality_p = 1000
        for para in parameter:
            parm_set_2 =  SPCA_SRotF_kfold(tree=100, depth=30, max_sample=0.7, batch_size=5, iteration=para ,seed = 47)
            print(para,normal_test(parm_set_2),parm_set_2)

            # 找出最好的參數
            if np.mean(parm_set_2) < best_rmse:
              best_para = para
              best_rmse = np.mean(parm_set_2)
            iter_rmse.append(parm_set_2)

            # 紀錄每個參數的常態檢定p-value
            iteration_data[str(para)].iloc[num] = normal_test(parm_set_2)

            if normal_test(parm_set_2) < min_normality_p:
              min_normality_p = normal_test(parm_set_2)
              
        if min_normality_p < 0.05 :
          test_stat,anova_p = kruskal(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('kruskal_oneway:',test_stat,anova_p)              
        else:
          test_stat,anova_p = f_oneway(iter_rmse[0],iter_rmse[1],iter_rmse[2],iter_rmse[3])
          print('f_oneway:',test_stat,anova_p)
  
        # 紀錄該參數的anova result
        anova_data['iteration'].iloc[num] = np.round(anova_p,3)

        # 紀錄該參數的最佳參數
        best_parameter['iteration'].iloc[num] = best_para

        # 紀錄每個參數之平均rmse
        iter_result = [np.mean(iter_rmse[0]),np.mean(iter_rmse[1]),np.mean(iter_rmse[2]),np.mean(iter_rmse[3])]

print(iter_result)

In [ ]:
depth_data.to_csv("depth.csv",index=False)
tree_data.to_csv("tree.csv",index=False)
bootstrap_size_data.to_csv("bootstrap size.csv",index=False)
batch_data.to_csv("batch size.csv",index=False)
iteration_data.to_csv("iteration.csv",index=False)

best_parameter.to_csv("bast parameter.csv",index=False)
anova_data.to_csv("anova result.csv",index=False)